<a href="https://colab.research.google.com/github/dntmdxor99/ML/blob/main/Deep_Learning/%EA%B3%BC%EC%A0%9C4_%EB%94%A5%EB%9F%AC%EB%8B%9D_4_Fashion_MNIST_(solution_TPU%EB%B2%84%EC%A0%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""과제4_딥러닝_4_Fashion_MNIST_(solution_GPU버전).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1L1m0yTx9vUTlkMuJJvnZ7ePam3JD08k8
"""

#!pip install tensorflow-determinism

import os
####*IMPORANT*: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED']=str(1)
import random

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD,Adam,Adagrad,RMSprop
print(tf.__version__)

2.8.0


In [2]:
### WARN: Must have ###
# to make this notebook's output stable across runs
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   tf.random.set_seed(1)
   np.random.seed(1)
   random.seed(1)
   os.environ['TF_DETERMINISTIC_OPS'] = '1'
   
reset_random_seeds()

# fashion MNIST 읽어 와서 신경망에 입력할 형태로 변환
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

4431872/4422102 [==============================] - 0s 0us/step


In [7]:
# 모델을 설계해주는 함수(모델을 나타내는 객체 model을 반환)
def run_model(x_train, y_train):    
    ####*IMPORANT*: Have to do this below line *every* re-run modeling & training
    reset_random_seeds()
    model=Sequential()
    model.add(Dense(512,activation='relu',input_shape=(784,)))
    model.add(Dense(512,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(),
                  metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=256, epochs=5, verbose=0)
    return model

In [4]:
### Update: 22.04.01
n_of_models = 3
acc = 0
models = []
if tf.test.is_gpu_available():
  for n in range (0, n_of_models):
    models.append(run_model(x_train, y_train))
    acc += models[n].evaluate(x_test,y_test,verbose=0)[1]*100
    print('Model {}th accuracy: {}'.format(n, models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc /= n_of_models
else:
  n_of_models = 1
  n = 0
  models.append(run_model(x_train, y_train))
  print('Model {}th accuracy: {}'.format(n, models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc =  models[n].evaluate(x_test,y_test,verbose=0)[1]*100

g_org_model_acc = acc
print('Baseline (average) accuracy: {}'.format(acc))

models[0].summary()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Model 0th accuracy: 86.75000071525574
Baseline (average) accuracy: 86.75000071525574
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [255]:
from tensorflow.python.ops.init_ops_v2 import Initializer
from tensorflow import keras

he = keras.initializers.HeNormal()
xavier = keras.initializers.GlorotNormal()
# 여기 함수를 수정하여 본인만의 새로운 모델을 만드시오
def run_proposed_model(x_train, y_train):  
    ####*IMPORANT*: Have to do this below line *every* re-run modeling & training
    reset_random_seeds()  
    model = Sequential()
    model.add(Dense(1024,activation='relu',input_shape=(784,), kernel_initializer=he))
    model.add(Dense(512, activation='relu', kernel_initializer=he))
    model.add(Dense(256, activation='relu', kernel_initializer=he))
    model.add(Dense(512, activation='relu', kernel_initializer=he))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(),
                  metrics=['accuracy'])
    
    # [※주의] 변경하면 안되는 옵션 : epochs, batch_size, _train, _text
    model.fit(x_train, y_train, batch_size=256, epochs=5, verbose=1)
    return model

In [256]:
### Update: 22.04.01

n_of_models = 3
acc = 0
pro_models = []
if tf.test.is_gpu_available():
  for n in range (0, n_of_models):
    pro_models.append(run_proposed_model(x_train, y_train))
    acc += pro_models[n].evaluate(x_test,y_test,verbose=1)[1]*100 # verbose=1 when you want to see the progress
    print('Model {}th accuracy: {}'.format(n, 
                                           pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc /= n_of_models
else:
  n_of_models = 1
  n = 0
  pro_models.append(run_proposed_model(x_train, y_train))
  print('Model {}th accuracy: {}'.format(n, 
                                         pro_models[n].evaluate(x_test,y_test,verbose=1)[1]*100))
  acc =  pro_models[n].evaluate(x_test,y_test,verbose=1)[1]*100 # verbose=1 when you want to see the progress

g_pro_model_acc = acc
print('Proposed (average) accuracy: {}'.format(acc))
  
# 내가 개선한 모델의 정확률을 기존과 비교하며 출력
print('Baseline vs. Mine: {} vs. {} under GPU {}'.format(g_org_model_acc, 
                                                         g_pro_model_acc, 
                                                         tf.test.is_gpu_available()))

Epoch 1/5
235/235 [==============================] - 13s 53ms/step - loss: 0.4962 - accuracy: 0.8214
Epoch 2/5
235/235 [==============================] - 12s 53ms/step - loss: 0.3487 - accuracy: 0.8712
Epoch 3/5
235/235 [==============================] - 13s 53ms/step - loss: 0.3118 - accuracy: 0.8849
Epoch 4/5
235/235 [==============================] - 13s 54ms/step - loss: 0.2869 - accuracy: 0.8928
Epoch 5/5
313/313 [==============================] - 2s 7ms/step - loss: 0.3415 - accuracy: 0.8769
Model 0th accuracy: 87.69000172615051
313/313 [==============================] - 2s 7ms/step - loss: 0.3415 - accuracy: 0.8769
Proposed (average) accuracy: 87.69000172615051
Baseline vs. Mine: 86.75000071525574 vs. 87.69000172615051 under GPU False
